# 1\_LinkedServer.ipynb

Written by Taiob  Ali

SqlWorldWide.com

This script will  

- Create linked server pointing to Azure SQL Database
- Collect file size from Azure SQL Database and save result on-premise
- Collect file size locally in Azure SQL database

<mark>If you have large number of Azure SQL Database might not be practical</mark>

Run this in your On-Premises SQL Server

In [ ]:
USE [master]
GO
--ALTER DATABASE [DbaDatabase] SET  SINGLE_USER WITH ROLLBACK IMMEDIATE
DROP DATABASE IF EXISTS DbaDatabase;
GO
CREATE DATABASE DbaDatabase;
GO

USE [DbaDatabase]
GO

SET ANSI_NULLS ON;
GO
SET QUOTED_IDENTIFIER ON;
GO

DROP TABLE IF EXISTS dbo.databaseSize;
GO
CREATE TABLE [dbo].[databaseSize]
(
    [collectedAT] [datetime] NOT NULL,
    [serverName] [nvarchar](128) NULL,
    [databaseName] [nvarchar](128) NULL,
    [fileName] [nvarchar](64) NULL,
    [fileId] [int] NOT NULL,
    [fileSizeMB] [int] NOT NULL,
    [spaceUsedMB] [numeric](12, 2) NULL,
    [freeSpaceMB] [numeric](12, 2) NULL,
    [percentFree] [numeric](12, 2) NULL,
    [physicalName] [nvarchar](260) NULL
) ON [PRIMARY];
GO


Creating a linked server in my on-premises SQL Server and pointig to Azure SQL Database

Read the password from JSON file  
How did I create this file?  
[https://dba.stackexchange.com/questions/263871/linked-server-from-on-premise-to-azure-sql-database/263881#263881](https://dba.stackexchange.com/questions/263871/linked-server-from-on-premise-to-azure-sql-database/263881#263881)

Open connections tab and show the linked server

In [ ]:
DECLARE @password VARCHAR(MAX)
SELECT @password = json_value(BulkColumn, '$[0]')
FROM OPENROWSET(BULK 'C:\password.json', SINGLE_CLOB) AS x
--Drop and create linked server
IF EXISTS(SELECT *
FROM sys.servers
WHERE name = N'AzureDB_adventureworks')
EXEC master.dbo.sp_dropserver @server=N'AzureDB_adventureworks', @droplogins='droplogins';
EXEC master.dbo.sp_addlinkedserver
 @server = N'AzureDB_adventureworks', 
 @srvproduct=N'',
 @provider=N'SQLNCLI',
 @datasrc=N'ugdemotargetserver.database.windows.net',
 @catalog=N'adventureworks';

EXEC master.dbo.sp_addlinkedsrvlogin
 @rmtsrvname=N'AzureDB_adventureworks',
 @useself=N'False',
 @rmtuser=N'taiob',@rmtpassword=@password;
GO

Collecting database file size from an Azure SQL Database and insert into local database

In [ ]:
INSERT INTO dbaDatabase.dbo.databasesize
SELECT *
FROM OPENQUERY 
(AzureDB_adventureworks, 
	'SELECT 
		GETDATE() AS collectedAT,
		@@SERVERNAME AS serverName, 
		DB_NAME() AS databaseName, 
		LEFT(a.name, 64) AS fileName,
		a.file_id AS fileId,
		a.size AS fileSizeMB,
		CONVERT(DECIMAL(12, 2), ROUND(FILEPROPERTY(a.name,''SpaceUsed'')/ 128.000, 2)) AS spaceUsedMB,
		CONVERT(DECIMAL(12, 2), ROUND(( a.size - FILEPROPERTY(a.name,''SpaceUsed''))/ 128.000, 2)) AS freeSpaceMB,
		CONVERT(DECIMAL(12, 2), (CONVERT(DECIMAL(12, 2), ROUND((a.size - FILEPROPERTY(a.name,''SpaceUsed''))/128.000, 2))*100)/ CONVERT(DECIMAL(12, 2), ROUND(a.size / 128.000, 2))) as percentFree,
		a.physical_name AS physicalName 
FROM adventureworks.sys.database_files a'
) ;
GO

<span style="font-size: 14px;">Look at the result</span>

In [ ]:
SELECT
  [collectedAT] AS [collectedAtUTC],
  DATEADD(mi, DATEDIFF(mi, GETUTCDATE(), GETDATE()), collectedAT) AS [collectedAtLocalTime],
  [serverName],
  [databaseName],
  [fileName],
  [fileId],
  [fileSizeMB],
  [spaceUsedMB],
  [freeSpaceMB],
  [percentFree],
  [physicalName]
FROM [DbaDatabase].[dbo].[databasesize]
ORDER BY collectedAT DESC;
GO

<span style="font-size: 14px;">If you want to save the result locally in Azure SQL Database</span>

<span style="font-size: 14px;">Confirm collection table exist</span>

Connect to ugdemotargetserver.database.windows.net adventureworks database

In [ ]:
DROP TABLE IF EXISTS dbo.databaseSize;
GO
CREATE TABLE [dbo].[databaseSize]
(
	[collectedAT] [datetime] NOT NULL,
	[serverName] [nvarchar](128) NULL,
	[databaseName] [nvarchar](128) NULL,
	[fileName] [nvarchar](64) NULL,
	[fileId] [int] NOT NULL,
	[fileSizeMB] [int] NOT NULL,
	[spaceUsedMB] [numeric](12, 2) NULL,
	[freeSpaceMB] [numeric](12, 2) NULL,
	[percentFree] [numeric](12, 2) NULL,
	[physicalName] [nvarchar](260) NULL
) ON [PRIMARY];
GO

Change connection back to local server  
Executing from On-Premises SQL Server but saving the result in Azure SQL Database

In [ ]:
INSERT INTO AzureDB_adventureworks.adventureworks.dbo.databasesize
SELECT *
FROM OPENQUERY 
(AzureDB_adventureworks, 
	'
	SELECT 
		GETDATE() AS collectedAT,
		@@SERVERNAME AS serverName, 
		DB_NAME() AS databaseName, 
		LEFT(a.name, 64) AS fileName,
		a.file_id AS fileId,
		a.size AS fileSizeMB,
		CONVERT(DECIMAL(12, 2), ROUND(FILEPROPERTY(a.name,''SpaceUsed'')/ 128.000, 2)) AS spaceUsedMB,
		CONVERT(DECIMAL(12, 2), ROUND(( a.size - FILEPROPERTY(a.name,''SpaceUsed''))/ 128.000, 2)) AS freeSpaceMB,
		CONVERT(DECIMAL(12, 2), (CONVERT(DECIMAL(12, 2), ROUND((a.size - FILEPROPERTY(a.name,''SpaceUsed''))/128.000, 2))*100)/ CONVERT(DECIMAL(12, 2), ROUND(a.size / 128.000, 2))) as percentFree,
		a.physical_name AS physicalName 
 FROM sys.database_files a'
) ;
GO

<span style="font-size: 14px;">Connect to ugdemotargetserver.database.windows.net adventureworks database</span>

<span style="font-size: 14px;">See the result</span>

In [ ]:
SELECT
    [collectedAT] AS [collectedAtUTC]
    , [serverName]
    , [databaseName]
    , [fileName]
    , [fileId]
    , [fileSizeMB]
    , [spaceUsedMB]
    , [freeSpaceMB]
    , [percentFree]
    , [physicalName]
FROM [dbo].[databasesize];
GO